# A Pythonic Object

## Object Representations

`repr()` - return a string representing the object as the developer wants to see it

`str()` - return a string representing the object as the user wants to see it

Implemented with the special methods `__repr__` and `__str__`

In [37]:
from array import array
import math
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) 
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
            bytes(array(self.typecode, self))) 
        
    def __eq__(self, __o: tuple) -> bool:
        return tuple(self) == tuple(__o)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

1. typecode is a class  attribute that is used when converting Vector2d instances to/from bytes
2. `__iter__` makes a Vector2d iterable, which makes tuple unpacking work
3. `__repr__` builds a string by interpolating the components with {!r} to get their repr; because Vector2d is iterable, *self feeds the x and y components to format.
4. In `__eq__`, to quickly compare all components, build tples out of the operands.

In [10]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [11]:
v1

Vector2d(3.0, 4.0)

In [12]:
print(v1)

(3.0, 4.0)


In [14]:
octets = bytes(v1) 
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [15]:
abs(v1)

5.0

In [16]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

## An Alternative Constructor 

Because we can export a Vector2d as bytes, naturally we need a method that imports a Vector2d from a binary sequence.

```py
@classmethod 
def frombytes(cls, octets): 
    typecode = chr(octets[0]) 
    memv = memoryview(octets[1:]).cast(typecode) 
    return cls(*memv)
```

1. Class method is modified by the classmethod decorator
2. No self argument; instead the class itself is passed as cls

## classmethod vs staticmethod

- `classmethod` changes the way the method is called,so it receives the class itself as the first argument, instead of an instance. Its most common use is for alternative constructors
- `staticmethod` decorator changes a method so that it receives no special first argument. In essence, a static method is just like a plain function that happens to live in a class body, instead of being defined at the module level

## formatted Displays


In [21]:
brl = 1/2.43
brl

0.4115226337448559

In [22]:
print(f"{brl: 0.4}")

 0.4115


In [23]:
format(42, 'b')

'101010'

In [24]:
format(2/3, '.1%')

'66.7%'

If a class has no __format__, the method inherited from object returns str(my_object)

In [25]:
format(v1, 0.3f)

SyntaxError: invalid syntax (<ipython-input-25-85c958a4338c>, line 1)

```py
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self) # 
    return '({}, {})'.format(*components)
 ```

## A Hashable Vector2d

Making x and y read only properties

In [36]:
class Vector2d:
    typecode = 'd'
    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    

1. Use two leading underscores to make an attribute private
2. The @property decorator marks the getter method of a property
3. The getter method is named after the public property is exposes: x

In [29]:
v1 = Vector2d(1,2)
v2 = Vector2d(1,2)
v3 = Vector2d(2,4)
v4 = Vector2d(4,2)

In [30]:
hash(v1), hash(v2), hash(v3), hash(v4)

(3, 3, 6, 6)

## Private and "Protected" Attributes in Python

Private attributes created with two leading underscores before the instance attribute name. Python stores the name in
the instance `__dict_`_ prefixed with a leading underscore and the class name. Name mangling is about safety, not security: it’s designed to prevent accidental access and not intentional wrongdoing 

In [31]:
v1 = Vector2d(3, 4)

In [32]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

## Saving Space with the `__slots__` Class Attribute

If you are dealing with millions of instances with few attributes, the
`__slots__` class attribute can save a lot of memory, by letting the interpreter store the instance attributes in a tuple instead of a dict.

To define `__slots__`, you create a class attribute with that name and assign it an iterable of str with identifiers for the instance attributes.

```py
class Vector2d:
    __slots__ = ('__x', '__y)
    ...
```

When `__slots__` is specified in a class, its instances will not be
allowed to have any other attributes apart from those named in
`__slots__`.

If you need the instances to be targets of weak referencesinclude '`__weakref__`' among the attributes named in `__slots__`.

It is mostly useful when working with tabular data such as database records where the schema is fixed by definition and the datasets may be very large. 

### The Problems with `__slots__`

To summarize, `__slots__` may provide significant memory savings if properly used, but there are a few caveats:

- You must remember to redeclare `__slots__` in each subclass, because the inherited attribute is ignored by the interpreter.
- Instances will only be able to have the attributes listed in `__slots__`, unless you include '`__dict__`' in `__slots__` (but doing so may negate the memory savings).
- Instances cannot be targets of weak references unless you remember to include '`__weakref__`' in `__slots__`.

## Overriding Class Attributes

But if you write to an instance attribute that does not exist, you create a new instance attribute—e.g., a typecode instance attribute—and the class attribute by the same name is untouched. However, from then on, whenever the code handling that instance reads self.typecode, the instance typecode will be retrieved, effectively shadowing the class
attribute by the same name. 

In [39]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [40]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [41]:
Vector2d.typecode

'd'

If you want to change a class attribute you must set it on the class directly, not through an instance.

Can also use subclassing to override the class attribute.

In [42]:
class ShortVector2d(Vector2d):
    typecode = 'f'
    
sv = ShortVector2d(1/11, 1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [43]:
len(bytes(sv))

9

# Chapter Summary

A Pythonic object should be as simple as the requirements allow—and not a parade of language features.

-  All string/bytes representation methods: `__repr__`, `__str__`, `__format__`, and `__bytes__`.
-  Several methods for converting an object to a number: `__abs__`, `__bool__`, `__hash__`.
- The `__eq__` operator, to test bytes conversion and to enable hashing (along with `__hash__`).
- also implemented an alternative constructor
- decorators @classmethod (very handy) and @staticmethod 
- looked at private an protected attributes, exposing them as read only properties
- looked at the benefits and caveats of `__slots__`